In [ ]:
import requests 
from bs4 import BeautifulSoup as bs
from openpyxl import load_workbook, Workbook
import urllib.request
import pandas as pd
import os

# cid(카테고리 id) 정의
CAT_ID = {'수납/정리': '019000000000000',
        '주방/욕실/청소': '020000000000000',
        '가구/인테리어': '022000000000000',
        '사무/문구/디지털': '021000000000000',
        '가전/레져/식품': '023000000000000',
        '키즈/뷰티/패션잡화': '024000000000000',
        '다이소 매장상품': '010000000000000',
        '포장재 전문관': '027000000000000'}

TARGET = '고무장갑'
INPUT_FILE_PATH = "../artifact/촬영 대상 물품 분류체계_v0.1_다이소몰 크롤링 목록_일반물품.xlsx"
OUTPUT_FILE_PATH = "%s_텍스트.xlsx" %TARGET
IMG_PATH = "C:\\Users\\user\\Desktop\\item\\%s" %TARGET
COLUMNS = ["물품분류", "물품코드", "물품종", "순번", "상품번호", "카테고리", "상품명", "상품사진", "가격", "링크"]
URL = "https://www.daisomall.co.kr/shop/search.php?nset=1&page={}&max=50&search_text={}&orderby=daiso_ranking1&cid={}&depth=1"
HEADERS = {'User-Agent':'Mozilla/5.0'}

def write_data():

    input_wb = load_workbook(INPUT_FILE_PATH)
    input_ws = input_wb[TARGET]
    row_max = input_ws.max_row

    if os.path.isfile(OUTPUT_FILE_PATH):
        output_wb = load_workbook(OUTPUT_FILE_PATH)
        output_ws = output_wb.create_sheet(TARGET)
    else:
        output_wb = Workbook()
        sheet_name = output_wb["Sheet"]
        sheet_name.title = TARGET

    output_ws.append(COLUMNS)

    for row in range(2, row_max+1):
        # Column 설명 -> 1.물품분류, 2.검색어, 3.포함 단어, 4. 제외 단어, 5. 비고, 6. 총 상품 개수, 7. 카테고리
        categories = []
        type = str(input_ws.cell(row, column=1).value) # 물품분류
        search = str(input_ws.cell(row, column=2).value) # 검색어
        
        if type == 'None': type = str(input_ws.cell(row-1, column=1).value)
        if search == 'None': continue

        search_include = str(input_ws.cell(row, column=3).value).split(', ')    # 포함 단어
        search_except = str(input_ws.cell(row, column=4).value).split(', ')     # 제외 단어
        # max_item = int(str(ws.cell(row=r, column=6).value)[5:-1])           # 총 상품 개수

        for column in range(7, input_ws.max_column+1):  # 카테고리명을 리스트에 저장
            category = str(input_ws.cell(row, column).value)

            if category == 'None': continue

            categories.append(category)

        print("검색어 : %s" %search)
        print("--------------------------")

        num = 1  # 순번
        for category in categories:
            index1 = category.find('(')
            index2 = category.find(')')
            itemNum = int(category[index1+1:index2].replace(',', ''))
            categoryPage = int(itemNum/50) + 1

            if categoryPage > 150: categoryPage = 150 # MAX 150 페이지

            category = category[:index1]

            try:
                cid = CAT_ID[category]  # 한글로 된 카테고리명을 url에서 쓰이는 cid로 변환
            except:
                print("unknown category : %s" %category)
                continue

            print("카테고리 명 : %s" %category)
            print("------------------------------")
            for page in range(1, categoryPage+1):
                print("Page : %s" %page)

                url = URL.format(page, search, cid)

                res = requests.get(url, headers=HEADERS)
                res.raise_for_status()

                soup = bs(res.text, "lxml")
                items = soup.find_all("li", {"class": "float01 search_goods_list"}) # 현 페이지 상품 리스트

                for item in items:
                    # "카테고리", "상품명", "상품사진", "가격", "링크"
                    title = item.find('div', {"style": "margin-top:10px;height:38px;"})
                    itemName = title.find('a').get("title")  # 상품명
                    
                    if itemName.find("<b>") == -1:  # 상품명에 검색어가 포함되지 않은 항목 제외
                        continue
                    if itemName.find("밀크북") != -1: # 모든 상품에 일괄적으로 제외할 단어들
                        continue
                    if itemName.find("양장") != -1:
                        continue

                    flag = 0
                    for word in search_include: # '포함 단어' 리스트에 있는 단어 체크
                        if itemName.find(word) == -1:
                            flag = 1
                            break

                    # '제외단어' 리스트에 있는 단어가 하나라도 포함된 경우 제외
                    for word in search_except:
                        if itemName.find(word) != -1:
                            flag = 1
                            break

                    if flag == 1:
                        continue

                    
                    itemName = itemName.replace("<b>", '').replace("</b>", '')

                    print(itemName)

                    price = item.find('div', {"style": "margin-top:12px;"})  # 상품가격
                    itemPrice = price.find('strong').text
                    itemPrice = itemPrice.replace("원", '')
                    itemPrice = itemPrice.replace(",", '')

                    img = item.find('div', {"class": "goods_line_img"})  # 상품 이미지 url
                    imgUrl = img.find('img').get('src')
                    imgName = "{}{}.jpg".format(search, num)  # 이미지 파일 이름 형식

                    # 상품 이미지 다운로드
                    urllib.request.urlretrieve(imgUrl, IMG_PATH+imgName)

                    itemId = item.find('a').get('href')  # 상품번호
                    itemId = itemId[24:34]

                    # 상품번호로 해당 상품의 상세페이지 url 접속해 http 소스 읽어옴
                    itemUrl = "https://www.daisomall.co.kr/shop/goods_view.php?id={}&depth=1&search_text={}".format(itemId, search)
                    try:
                        itemRes = requests.get(itemUrl, headers={'User-Agent':'Mozilla/5.0'})
                    except:
                        print("request error!")
                        continue
                    itemRes.raise_for_status()
                    soupItem = bs(itemRes.text, "lxml")

                    try:
                        itemNum = soupItem.find('td', {"class": "color_63 line_h160"}).find('strong').text 
                    except:
                        print("itemNum error!")
                        continue
                    
                    itemCategories = []
                    for itemCategory in soupItem.find_all('option', {"selected": ""}):  # 해당 상품의 카테고리 저장
                        if str(itemCategory).find("selected") == -1:
                            continue
                        itemCategory = itemCategory.get_text()
                        itemCategories.append(itemCategory)

                    try:
                        data = [type, None, search, num, int(itemNum), itemCategories[0]+">"+itemCategories[1]+">"+itemCategories[2], itemName, imgUrl, int(itemPrice), itemUrl]
                    except:
                        print("category error!")
                        continue
                    output_ws.append(data)
                    num+=1
                    
            output_wb.save(OUTPUT_FILE_PATH)

def drop_duplicates():
    df = pd.read_excel(OUTPUT_FILE_PATH)
    df = df.drop_duplicates(subset='상품번호')
    df.to_excel(OUTPUT_FILE_PATH[:-5]+"_final.xlsx")

def load_code():
    input_wb = load_workbook(INPUT_FILE_PATH)
    input_ws = input_wb['물품코드표']
    code_row_max = input_ws.max_row

    output_wb = load_workbook(OUTPUT_FILE_PATH)
    output_ws = output_wb.active
    row_max = output_ws.max_row
    
    codeDic = {'item': 'code'}
    for r in range(2, code_row_max+1):
        code = str(input_ws.cell(row=r, column=1).value)
        item = str(input_ws.cell(row=r, column=2).value)
        desc = str(input_ws.cell(row=r, column=3).value)
        if desc.find("삭제") != -1:
            continue
        codeDic[item] = code

    for r in range(2, row_max+1):
        temp = str(output_ws.cell(row=r, column=1).value)
        try:
            output_ws.cell(row=r, column=2).value = int(codeDic[temp])
        except:
            print("Key error! --- " + temp)
            continue
        
    output_wb.save(OUTPUT_FILE_PATH)

write_data()
# load_code() # 물품코드 로딩
# drop_duplicates() # 중복 행 제거(상품 번호 기준)